In [3]:
from pathlib import Path
import requests
import json
from configparser import ConfigParser

import psycopg
import pandas as pd
import geopandas as gpd

In [4]:
parser = ConfigParser()
# read config file
parser.read("database.ini")
params = dict(parser["postgresql"])


# Set up database

Schema: 

![Schema](../data/Schema.png)

In [13]:
with psycopg.connect(**params) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Execute a command: this creates a new table
        cur.execute("""
            CREATE TABLE Countries (
                ISO text PRIMARY KEY,
                EnglishName text,
                Endonym text,
                ADM0simple geometry)
            """)
        cur.execute("""
            CREATE TABLE ADM1 (
                Country_ISO text,
                FOREIGN KEY (Country_ISO) REFERENCES Countries(ISO)
                ADM1_ID Int PRIMARY KEY
                ADM1simple geometry)
            """)
        cur.execute("""
            CREATE TABLE Organizations (
                Org_ID text PRIMARY KEY,
                OrgName text
            );
            """)
        cur.execute("""
            CREATE TABLE Memberships (
                Country_ISO text,
                Org_ID text,
                PRIMARY KEY (Country_ISO, Org_ID),
                FOREIGN KEY (Country_ISO) REFERENCES Countries(ISO),
                FOREIGN KEY (Org_ID) REFERENCES Organizations(Org_ID)
            );
            """)

In [94]:
with psycopg.connect(**params) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        cur.execute("""
            CREATE TABLE Organizations (
                Org_ID text PRIMARY KEY,
                OrgName text
            );
            """)
        cur.execute("""
        CREATE TABLE Memberships (
            Country_ISO text,
            Org_ID text,
            PRIMARY KEY (Country_ISO, Org_ID),
            FOREIGN KEY (Country_ISO) REFERENCES Countries(ISO),
            FOREIGN KEY (Org_ID) REFERENCES Organizations(Org_ID)
        );
        """)

In [49]:
organizations = pd.read_csv("../data/CountrycodesOrgs.csv")

In [77]:
orgs = organizations[organizations.org_member == "member"].groupby("org_id")

In [92]:
org_dicts = {}

for name, group in orgs:
    org_dicts.update({name: dict(
        org_name = group.org_name.values[0].strip(),
        members = list(group.ISO_3166_3.values)
    )})

In [93]:
with open("../data/orgs.json", "w+") as fs:
    json.dump(org_dicts, fs, indent=2)

In [103]:

with psycopg.connect(**params, autocommit=True) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        for org_short, values in org_dicts.items():
            cur.execute("INSERT INTO Organizations (Org_ID, OrgName) VALUES (%s, %s)",
                        (org_short, values["org_name"]))
            for member in values["members"]:
                try:
                    cur.execute("INSERT INTO Memberships (Org_ID, Country_ISO) VALUES (%s, %s)",
                            (org_short, member))
                except psycopg.errors.ForeignKeyViolation:
                    print(f"{member} in Org {values['org_name']} not a country in Countries table.")
                    conn.rollback()

HKG in Org Asian Development Bank not a country in Countries table.
HKG in Org Asia-Pacific Economic Cooperation not a country in Countries table.
ESH in Org African Union not a country in Countries table.
HKG in Org Bank for International Settlements not a country in Countries table.
HKG in Org Financial Action Task Force not a country in Countries table.
HKG in Org International Hydrographic Organization not a country in Countries table.
MAC in Org International Hydrographic Organization not a country in Countries table.
SXM in Org International Labor Organization not a country in Countries table.
HKG in Org International Monetary Fund not a country in Countries table.
MAC in Org International Monetary Fund not a country in Countries table.
SXM in Org International Criminal Police Organization not a country in Countries table.
HKG in Org International Olympic Committee not a country in Countries table.
PRI in Org International Olympic Committee not a country in Countries table.
TKL i

In [101]:
with psycopg.connect(**params) as conn:
    conn.rollback()

That next boy gets a single row of the table and returns its geometry as a geojson. Sweet as.

In [44]:
with psycopg.connect(**params) as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        cur.execute(
            """
SELECT jsonb_build_object(
'type',       'Feature',
'id',         iso,
'geometry',   ST_AsGeoJSON(simplegeom)::jsonb,
'properties', to_jsonb(row) - 'iso' - 'simplegeom'
) FROM (SELECT * FROM adm0) row limit 1;"""
        )
        for thing in cur:
            print(thing)


({'id': 'BEL', 'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[5.10218, 51.429005], [5.081105, 51.466647], [5.049276, 51.472331], [5.03382, 51.484649], [5.014763, 51.474288], [5.000885, 51.44337], [4.92708, 51.396821], [4.884981, 51.414596], [4.785251, 51.411103], [4.773403, 51.416937], [4.772836, 51.428643], [4.826744, 51.425198], [4.825754, 51.446873], [4.83904, 51.478104], [4.806476, 51.49684], [4.759926, 51.502464], [4.750611, 51.497989], [4.746145, 51.489613], [4.728776, 51.483376], [4.715785, 51.469155], [4.704348, 51.466608], [4.692802, 51.452317], [4.667286, 51.444439], [4.669544, 51.426384], [4.577543, 51.431913], [4.53833, 51.424937], [4.531711, 51.445505], [4.542871, 51.476821], [4.478998, 51.477104], [4.392661, 51.450081], [4.396215, 51.432564], [4.387892, 51.415014], [4.427319, 51.377158], [4.427162, 51.365743], [4.375975, 51.355481], [4.344422, 51.359063], [4.32898, 51.376562], [4.279565, 51.376017], [4.218264, 51.373026], [4.226164, 51.360486],

In [14]:
country_paths = list(Path("../geoBoundaries/releaseData/gbOpen/").iterdir())

In [15]:
def build_donwload_url(adm, iso):
    return f"https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/gbOpen/{iso}/{adm}/geoBoundaries-{iso}-{adm}_simplified.geojson"

The next cell adds the geometries for ADM0 to the database. Some might fail due to no file available

In [35]:
with psycopg.connect(autocommit=True, **params) as conn:
    with conn.cursor() as cur:
        for country_path in country_paths[195:]:
            download_url = build_donwload_url("ADM0", country_path.stem)
            r = requests.get(download_url)
            try:
                r.raise_for_status()
            except requests.HTTPError as e:
                if r.status_code == 404:
                    pass
                else:
                    raise e
            geoms = gpd.read_file(r.text)
            print(country_path.stem)
            for geom in geoms.itertuples():
                to_postgis = geom.geometry.wkb_hex
                cur.execute("INSERT INTO Countries (ISO, EnglishName) VALUES (%s, %s)",
                            (country_path.stem, geom.shapeName))
                cur.execute("INSERT INTO ADM0 (ISO, SimpleGeom) VALUES (%s, ST_SetSRID(%s::geometry, %s))",
                        (country_path.stem, to_postgis, 4326))

SLE
ERI
ESP
SVK
TUV
CZE
KEN
SHN
VGB
SLB
COL
SAU
MAR
HND
KOR
KHM
SUR
GEO
PCN
CHN
GUM
FLK
ZWE
ASM
YEM
TLS
LSO
MCO
ATG
GMB
MTQ
URY
GRL
CRI
PER
GRC


In [18]:
geoms = gpd.read_file(r.text)
geom

Pandas(Index=0, shapeName='The Philippines', shapeISO='', shapeID='24100683B85265433280220', shapeGroup='PHL', shapeType='ADM0', geometry=<MULTIPOLYGON (((119.469 4.594, 119.468 4.594, 119.468 4.59, 119.466 4.587, ...>)